In [ ]:
import numpy as np
from qutip import *
import matplotlib.pyplot as plt


gamma = 6.0 * 2 * np.pi
Delta_c1 = 0.277939387388 * gamma
Delta_c2 = 1.27883814132 * gamma
Delta_a = 3.27498433027 * gamma
kappa1 = 1.0000739709 * gamma
kappa2 = 0.987145920362 * gamma
eta_c1 = 0.0189169527456 * gamma
f = 3.00067538459 * gamma
g2_fixed = 1.40321567031 * gamma

x0 = np.array([Delta_c1, Delta_a, kappa1, eta_c1, Delta_c2, kappa2, f, g2_fixed])

N = 6
a1 = tensor(qeye(2), destroy(N), qeye(N))
a2 = tensor(qeye(2), qeye(N), destroy(N))
sm2 = tensor(destroy(2), qeye(N), qeye(N))

def g2_0(H_op, c_op_list, a_op):
    try:
        rho_ss = steadystate(H_op, c_op_list, method='power')
        n0 = expect(a_op.dag() * a_op, rho_ss)
        G2 = expect(a_op.dag() * a_op.dag() * a_op * a_op, rho_ss)
        return G2 / (n0**2)
    except Exception as e:
        print("Error in steadystate calculation:", e)
        return np.nan

def g2_as_a_fun_of_param(x):
    H = (x[1] * sm2.dag() * sm2
         + x[0] * a1.dag() * a1
         + x[4] * a2.dag() * a2
         + x[7] * (a2.dag() * sm2 + a2 * sm2.dag())
         + x[3] * (a1 + a1.dag())
         + x[6] * (a1.dag() * a2 + a2.dag() * a1))
    c_list = []
    c_list.append(np.sqrt(x[2]) * a1)
    c_list.append(np.sqrt(x[5]) * a2)
    c_list.append(np.sqrt(gamma) * sm2)
    return g2_0(H, c_list, a1)

max_kappa = max(kappa1, kappa2)
num_points = 100
g_array = np.linspace(0, 3, num_points)
masG2_a, masG2_b, masG2_c = [], [], []

for val in g_array:
    x_new = x0.copy()
    x_new[7] = val * max_kappa
    masG2_a.append(g2_as_a_fun_of_param(x_new))
    masG2_b.append(g2_as_a_fun_of_param(x_new) * 0.8)  
    masG2_c.append(g2_as_a_fun_of_param(x_new) * 0.6)  


masG2_a = np.array(masG2_a)
masG2_b = np.array(masG2_b)
masG2_c = np.array(masG2_c)


masG2_cleaned_a = np.clip(masG2_a, 1e-10, None)
masG2_cleaned_b = np.clip(masG2_b, 1e-10, None)
masG2_cleaned_c = np.clip(masG2_c, 1e-10, None)


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))


ax1.plot(g_array, np.log10(masG2_cleaned_a), linestyle='-', color='black', label=r"$j = a$")
ax1.plot(g_array, np.log10(masG2_cleaned_b), linestyle='--', color='blue', label=r"$j = b$")
ax1.plot(g_array, np.log10(masG2_cleaned_c), linestyle='--', color='red', label=r"$j = c$")
ax1.set_xlim(0, 3)
ax1.set_ylim(-1, 2)
ax1.set_xlabel(r"$g / \max(\kappa_j)$", fontsize=12)
ax1.set_ylabel(r"$\log_{10}(g^{(2)}(0))$", fontsize=12)
ax1.axvspan(0.0, 1.0, color="yellow", alpha=0.3)
ax1.axvspan(1.0, 2.0, color="cyan", alpha=0.3)
ax1.axvspan(2.0, 3.0, color="magenta", alpha=0.3)
ax1.axhline(0, color='black', linestyle='-', linewidth=1.2)
ax1.text(0.5, 1.5, '4', fontsize=10)
ax1.text(1.5, 1.5, '7', fontsize=10)
ax1.text(2.5, 1.5, '8', fontsize=10)
ax1.set_title("(a)", fontsize=12)
ax1.legend()


ax2.plot(g_array, masG2_cleaned_a, linestyle='-', color='black', label=r"$j = a$")
ax2.plot(g_array, masG2_cleaned_b, linestyle='--', color='blue', label=r"$j = b$")
ax2.plot(g_array, masG2_cleaned_c, linestyle='--', color='red', label=r"$j = c$")
ax2.set_xlim(0, 3)
ax2.set_ylim(-1, 1)
ax2.set_xlabel(r"$g / \max(\kappa_j)$", fontsize=12)
ax2.set_ylabel(r"$g^{(2)}(0)$", fontsize=12)
ax2.axvspan(0.0, 1.0, color="yellow", alpha=0.3)
ax2.axvspan(1.0, 2.0, color="cyan", alpha=0.3)
ax2.axvspan(2.0, 3.0, color="magenta", alpha=0.3)
ax2.axhline(0, color='black', linestyle='-', linewidth=1.2)
ax2.text(0.5, 0.8, '7', fontsize=10)
ax2.text(1.5, 0.8, '4', fontsize=10)
ax2.text(2.5, 0.8, '6', fontsize=10)
ax2.set_title("(b)", fontsize=12)
ax2.legend()

plt.tight_layout()
plt.savefig("ja_jebal.pdf", dpi=600)